Este notebook tem como objetivo demonstrar a leitura de arquivos CSV armazenados no DBFS (Databricks File System) e sua conversão em tabelas Delta para compor a camada Bronze do pipeline de dados.

Os arquivos utilizados foram previamente carregados para o diretório `/FileStore/tables/` e representam dados do e-commerce brasileiro da Olist. A leitura será feita utilizando PySpark, com inferência de schema e tratamento básico para garantir a consistência dos dados.

As tabelas criadas aqui serão utilizadas nas próximas etapas do pipeline (camada Silver), conforme definido no desafio técnico.


In [0]:
from pyspark.sql.functions import to_timestamp, col 

In [0]:
# Arquivos
path_orders = "/FileStore/tables/olist_orders_dataset.csv"
path_order_items = "/FileStore/tables/olist_order_items_dataset.csv"
path_customers = "/FileStore/tables/olist_customers_dataset.csv"

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS bronze;

# BRONZE

## olist_orders

In [0]:
# Vamos analisar os dados antes de salvar no Delta para verificar se precisa de alterações.

# Leitura do arquivo de pedidos
df_orders = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load(path_orders)

# display(df_orders) 
df_orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [0]:
# Olhando o root, as colunas de data/hora foram corretamente inferidas como timestamp pelo Spark, portanto, nenhuma conversão adicional foi necessária nesta etapa. Então vamos seguir para a escrita da tabela Delta no schema bronze

In [0]:
# Tabela olist_orders
df_orders.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze.olist_orders")

## olist_order_items

In [0]:
# Leitura do arquivo de itens de pedidos
df_order_items = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load(path_order_items)

# display(df_order_items)
df_order_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [0]:
# Realizei a conversão dos campos `price` e `freight_value` de `double` para `decimal(10,2)` para garantir maior precisão. Essa alteração também facilita a análise e agregações futuras, especialmente para o time de BI.

df_order_items_clean = df_order_items \
    .withColumn("price", col("price").cast("decimal(10,2)")) \
    .withColumn("freight_value", col("freight_value").cast("decimal(10,2)"))

In [0]:
# Tabela olist_order_items
df_order_items.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze.olist_order_items")

## olist_customers

In [0]:
# Leitura do arquivo de clientes
df_customers = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load(path_customers)

# display(df_customers)
df_customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [0]:
# Converti a coluna `customer_zip_code_prefix` para `string` para preservar possíveis zeros à esquerda e garantir consistência no tratamento como dado categórico.

df_customers_clean = df_customers.withColumn("customer_zip_code_prefix", col("customer_zip_code_prefix").cast("string"))

In [0]:
# Tabela olist_customers
df_customers_clean.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze.olist_customers")